In [1]:
import pickle
import pandas as pd
import cobra
from collections import defaultdict, OrderedDict, Counter
from cobra import Reaction, Metabolite, Model, Gene
import time
from glob import glob

from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.patches as mpatches
import sympy
from sympy import to_dnf, Add
from Bio import SeqIO
import re
import os
import scipy
import urllib
import scipy.stats as stats
from statsmodels.stats.multitest import *

import sys
sys.path.append("/home/yara/Documents/PseudoFind")
from PseudoFind.pangenome_cmds import *
from PseudoFind.functional_annotations import *

%matplotlib inline

#2 cluster pangenome into gene families using CD-HIT

In [2]:
# FT_LRI = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/FT_LRI.csv', dtype = {'Genome ID':str}, index_col = ['Unnamed: 0'])
# fasta_files = FT_LRI['Directory']
# pangenome_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/pangenome/staphylococcus_pangenome.fa'
# threshold = 0.8
# run_cdhit(fasta_files, pangenome_directory, threshold)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/yara/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-32044aeccef2>", line 5, in <module>
    run_cdhit(fasta_files, pangenome_directory, threshold)
  File "/home/yara/Documents/PseudoFind/PseudoFind/pangenome_cmds.py", line 289, in run_cdhit
    y.write(f.read())
  File "/usr/lib/python3.5/codecs.py", line 318, in decode
    def decode(self, input, final=False):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yara/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):

KeyboardInterrupt: 

In [3]:
# # add metadata to each gene family

# cdhit_clusters = get_cdhit_clusters('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/pangenome/staphylococcus_pangenome_cdhit.clstr')
# cdhit_clusters_rev = {y:x for x,z in cdhit_clusters.items() for y in z}

# gb_files = [x.replace('.faa', '.gbk').replace('_m','') for x in FT_LRI['Directory']]

# cdhit_metadata = {}
# for gb_file in gb_files:
    
#     FT = get_FT_from_gb(gb_file).set_index('locus_tag')

#     for index in FT.index:
#         cdhit_id = cdhit_clusters_rev[index]
#         if cdhit_id not in cdhit_metadata.keys():
#             cdhit_metadata[cdhit_id] = defaultdict(list)

#         for column in ['EC_number', 'gene', 'product']:
#             cdhit_metadata[cdhit_id][column].append(FT.loc[index, column])

KeyboardInterrupt: 

In [31]:
# cdhit_metadatar = {}
# for cdhit_id, metadata in cdhit_metadata.items():
#     product = list(OrderedDict(sorted(Counter(metadata['product']).items(), key = lambda a: a[1], reverse = True )).keys())[0]
#     if product == 'hypothetical protein' and len(set(metadata['product']) - {'hypothetical protein'}) != 0:
#         counter = Counter(metadata['product'])
#         del counter['hypothetical protein']
#         product = list(OrderedDict(sorted(counter.items(), key = lambda a: a[1], reverse = True )).keys())[0]
        
#     cdhit_metadatar[cdhit_id] = {
#         'EC_number':list(OrderedDict(sorted(Counter(metadata['EC_number']).items(), key = lambda a: a[1], reverse = True )).keys())[0],
#          'gene':list(OrderedDict(sorted(Counter(metadata['gene']).items(), key = lambda a: a[1], reverse = True )).keys())[0],
#          'product':product
#         }
    
# pickle.dump(cdhit_metadatar, open('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/pangenome/cdhit_metadatar.p', 'wb'))

#4 Map the allele matrix to the pan genome matrix

In [32]:
# pangenome_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/pangenome/staphylococcus_pangenome.fa'
# cdhit_directory = pangenome_directory.split('.f')[0]+'_cdhit.clstr'
# cdhit_clusters = get_cdhit_clusters(cdhit_directory)
# cdhit_clusters_rev = {y:x for x,z in cdhit_clusters.items() for y in z}

# output_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/allele_matrix/'
# locus2allele = pickle.load(open('%s/locus2allele.p'%output_directory, 'rb'))
# allele2cdhit = {allele_id:cdhit_clusters_rev[gene_id] for gene_id, allele_id in locus2allele.items()}
# pickle.dump(allele2cdhit, open('%s/allele2cdhit.p'%output_directory, 'wb'))

#5 Deep functional annotation of selected features - restart from here

In [57]:
selected_features_final = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/machine_learning/selected_features_final_trial_2.csv').rename(columns = {'Unnamed: 0':'Allele ID'})
selected_features_final['Allele ID'] = ['Allele_%d'%i for i in selected_features_final['Allele ID']]
output_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/allele_matrix/'
allele2cdhit = pickle.load(open('%s/allele2cdhit.p'%output_directory, 'rb'))

all_alleles = pickle.load(open('%s/all_alleles_seqs.p'%output_directory, 'rb'))
all_alleles_rev = {y:x for x,y in all_alleles.items()}

selected_features_final['CD-hit ID'] = [allele2cdhit[c] for c in selected_features_final['Allele ID']]
selected_features_final['Allele sequence'] = [all_alleles_rev[x] for x in selected_features_final['Allele ID']]

with open('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/machine_learning/selected_features_final.fa', 'w') as y:
    for index in selected_features_final.index:
        y.write('>'+selected_features_final.loc[index, 'Allele ID'] + '\n' + selected_features_final.loc[index, 'Allele sequence'] + '\n')
        
        
cdhit_metadatar = pickle.load(open('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/pangenome/cdhit_metadatar.p', 'rb'))
cdhit_metadatar = pd.DataFrame(cdhit_metadatar).T
selected_features_final = selected_features_final.merge(cdhit_metadatar, left_on = 'CD-hit ID', right_index = True, how = 'left')
selected_features_final.head(2)

,Allele ID,score,CD-hit ID,Allele sequence,EC_number,gene,product
0,Allele_66792,-129.278267,Cluster 1334,MTTVLFVGLGLIGGSLASNIKYHNPNTNIIAYDADTSQLDKAKSIG...,1.5.1.2,proC_1,Pyrroline-5-carboxylate reductase
1,Allele_76691,116.752713,Cluster 181,MDIGKKHVIPKSQYRRKRREFFHNEDREENLNQHQDKQNIDNTTSK...,3.4.21.-,NaN,Serine protease HtrA-like protein


#5a VFDB

In [58]:
directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/functional_annotation'
output_fa,output_ffn, output_csv = '%s/VFDB_staphylococcus.fa'%directory, '%s/VFDB_staphylococcus.ffn'%directory, '%s/VFDB_staphylococcus.csv'%directory
species_name = 'staphylococcus'
get_VFDB_species(species_name, output_fa = output_fa, output_ffn = output_ffn, output_csv = output_csv)


selected_features_fa = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/machine_learning/selected_features_final.fa'
blast_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/pangenome/BR_results'
run_bidirectional_blast(selected_features_fa, [output_fa], blast_directory)

PID_threshold = 80
output_file_name = 'SF_VFDB'
read_blast_results(blast_directory, PID_threshold, output_file_name)

bbh = pickle.load(open('%s/%s.p'%(blast_directory, output_file_name), 'rb'))
selected_features_final['VFDB hit'] = [bbh['selected_features_final'][x] if x in bbh['selected_features_final'].keys() else '' for x in selected_features_final['Allele ID']]

VFDB_met = pd.read_csv(output_csv)
VFDB_met = VFDB_met.rename(columns = {col:'%s(VFDB)'%col for col in VFDB_met.columns})
selected_features_final = selected_features_final.merge(VFDB_met, left_on = 'VFDB hit', right_on = 'gene_id(VFDB)', how = 'left')
selected_features_final.head(2)

/home/yara/.local/lib/python3.5/site-packages/Bio/Seq.py:2609 BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.


A total of 713 genes were extracted for staphylococcus
A total of 9 entries were excluded
Blasting: VFDB_staphylococcus...


,Allele ID,score,CD-hit ID,Allele sequence,EC_number,gene,product,VFDB hit,VF_id(VFDB),description(VFDB),gene_id(VFDB),gene_name(VFDB),nucleotide_sequence(VFDB),organism(VFDB)
0,Allele_66792,-129.278267,Cluster 1334,MTTVLFVGLGLIGGSLASNIKYHNPNTNIIAYDADTSQLDKAKSIG...,1.5.1.2,proC_1,Pyrroline-5-carboxylate reductase,,NaN,NaN,NaN,NaN,NaN,NaN
1,Allele_76691,116.752713,Cluster 181,MDIGKKHVIPKSQYRRKRREFFHNEDREENLNQHQDKQNIDNTTSK...,3.4.21.-,NaN,Serine protease HtrA-like protein,,NaN,NaN,NaN,NaN,NaN,NaN


#5b EGGNOG

In [59]:
eggnog_file = '%s/selected_features_final.fa.emapper.annotations'%directory
eggnog = get_eggnog(eggnog_file)
selected_features_final = selected_features_final.merge(eggnog, left_on = 'Allele ID', right_on = 'locus_tag', how = 'left')
selected_features_final.head(2)

,Allele ID,score_x,CD-hit ID,Allele sequence,EC_number,gene,product,VFDB hit,VF_id(VFDB),description(VFDB),...,GO,kegg,bigg,bactNOG,bactNOG_desc,path,COG cat,description,Cog cat description,COG (grouped)
0,Allele_66792,-129.278267,Cluster 1334,MTTVLFVGLGLIGGSLASNIKYHNPNTNIIAYDADTSQLDKAKSIG...,1.5.1.2,proC_1,Pyrroline-5-carboxylate reductase,,NaN,NaN,...,NaN,"K00210,K00220,K00800,K04517,K15226",PPND,bactNOG[38],"05EVN@bactNOG,0QK0X@gproNOG,16SIV@proNOG,COG02...",0QK0X|2.9e-76|258.4,E,Prephenate dehydrogenase,Amino acid metabolism and transport,Metabolism
1,Allele_76691,116.752713,Cluster 181,MDIGKKHVIPKSQYRRKRREFFHNEDREENLNQHQDKQNIDNTTSK...,3.4.21.-,NaN,Serine protease HtrA-like protein,,NaN,NaN,...,"GO:0000271,GO:0003674,GO:0003824,GO:0004175,GO...","K04771,K08372",NaN,bactNOG[38],"05C0H@bactNOG,0QHTY@gproNOG,16PJ0@proNOG,COG02...",0QHTY|6.6e-64|217.0,O,protease,Molecular chaperones and related functions,Cellular Processes & Signaling


#5c metabolism (GEM)

In [60]:
# PA14_model = cobra.io.load_json_model('%s/iPAU1129.json'%directory)

FT = get_FT_from_gb('/home/yara/Documents/UO1/Data/Sequences/gb/NC_010079.1_full.gbff').fillna('').set_index('locus_tag')
with open('%s/TCH1516.fa'%directory, 'w') as y:
    for index in FT.index:
        if FT.loc[index, 'translation'] != '':
            y.write('>'+index+'\n'+FT.loc[index, 'translation'] + '\n')
        
selected_features_fa = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/machine_learning/selected_features_final.fa'
blast_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/pangenome/BR_results'
output_fa = ['%s/TCH1516.fa'%directory]
run_bidirectional_blast(selected_features_fa, output_fa, blast_directory)

PID_threshold = 80
output_file_name = 'SF_TCH1516'
read_blast_results(blast_directory, PID_threshold, output_file_name)

bbh = pickle.load(open('%s/%s.p'%(blast_directory, output_file_name), 'rb'))

Blasting: TCH1516...


In [61]:
model = cobra.io.load_json_model('/home/yara/Documents/UO1/Intermediate/Reconstruction/jsons/USA300_TCH1516_v75.json')
gene_iYS854 = pd.DataFrame([{
                    'locus_tag':gene.id, 
                     'Subsystem(s)':' AND '.join({rx.subsystem for rx in gene.reactions}),
                    'Module(s)':' AND '.join({rx.notes['Module'] for rx in gene.reactions if 'Module' in rx.notes.keys()}),
                      'Reaction(s)':' AND '.join({rx.name for rx in gene.reactions}),
                    'Allele ID':bbh['TCH1516'][gene.id],
    'gene_name(model)':gene.name
                                 }
        for gene in model.genes if gene.id in bbh['TCH1516'].keys()
])
selected_features_final = selected_features_final.merge(gene_iYS854, on = 'Allele ID', how = 'left')


# add nucleotide sequences

In [62]:
FTSA = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/FT_LRI.csv', dtype = {'Genome ID':str}).set_index('Genome ID')
directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/'
locus2allele = pickle.load(open('%s/allele_matrix/locus2allele.p'%directory, 'rb'))
locus2allele_rev = {y:x for x,y in locus2allele.items()}
for index in selected_features_final.index:
    allele_id = selected_features_final.loc[index, 'Allele ID']
    locus_tag = locus2allele_rev[allele_id]
    
    ffn_file = FTSA.loc['_'.join(locus_tag.split('_')[:-1]), 'Directory'].replace('.faa','.ffn').replace('_m','')
    with open(ffn_file, 'r') as f:
        string = f.read()
    gene_seqs = {x.split('\n')[0].replace('>','').split(' ')[0]:''.join(x.split('\n')[1:]) for x in string.split('\n>')}    
    selected_features_final.loc[index, 'allele_nucleotide_sequence'] = gene_seqs[locus_tag]

In [63]:
selected_features_final = selected_features_final.fillna('')
selected_features_final.to_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/functional_annotation/selected_features_final_m.csv')

In [38]:
directory = '/home/yara/Documents/VISA_anaelle/data/'
FT = get_FT_from_gb('%s/%s.gbff'%(directory, 'USA300_FPR3757')).fillna('')
mutL_CF = FT.loc[FT['gene'] == 'mutL']['Nucleotide_sequence'].iloc[0]
mutL_ref = selected_features_final.loc[selected_features_final['gene'] == 'mutL']['allele_nucleotide_sequence'].iloc[0]

In [54]:
FT.loc[FT['gene'] == 'mutL']

,EC_number,EC_number_1,Nucleotide_sequence,codon_start,contig,end,gene,gene_synonym,inference,locus_tag,note,old_locus_tag,product,protein_id,pseudo,ribosomal_slippage,start,strand,transl_table,translation
1252,,,ATGGGGAAAATTAAAGAACTCCAAACCTCATTAGCAAATAAAATCG...,1,NC_007793.1,1311291,mutL,,COORDINATES: similar to AA sequence:RefSeq:YP_...,SAUSA300_RS06425,Derived by automated computational analysis us...,SAUSA300_1189,DNA mismatch repair endonuclease MutL,WP_000516261.1,,,1309281,1,11,MGKIKELQTSLANKIAAGEVVERPSSVVKELLENAIDAGATEISIE...


In [44]:
from PseudoFind.lof_detection import *

In [48]:
sequences = {'CF':mutL_CF, 'ref':mutL_ref}
# get_all_mutations('ref', sequences)
point_mutations, deletions, insertions = get_alignment_summary('CF','ref', sequences)
point_mutations, deletions, insertions

(['G1569A',
  'A1956T',
  'G714A',
  'A1035G',
  'C942T',
  'T1681C',
  'A1145C',
  'TG1335CA',
  'G1368A',
  'A1209G',
  'T1242C',
  'T1371C',
  'A1020G'],
 {786: 'C', 949: 'T', 1340: 'A', 1454: 'A', 1852: 'C'},
 {788: 'T', 951: 'A', 1342: 'T', 1456: 'C', 1854: 'T'})

In [51]:
sequences = {'CF':Seq(mutL_CF).translate(), 'ref':Seq(mutL_ref).translate()}
# get_all_mutations('ref', sequences)
point_mutations, deletions, insertions = get_alignment_summary('CF','ref', sequences)
point_mutations, deletions, insertions 

(['E382A', 'GT446DS'], {}, {})

In [53]:
str(Seq(mutL_CF).translate())[382]

'E'

### virulence

In [9]:
cols = ['score_x','gene', 'product', 'description', 'description(VFDB)', 'Cog cat description']
indices = set(selected_features_final.loc[selected_features_final['VF_id(VFDB)'] != ''].index.tolist()) 
selected_features_final.loc[indices][cols]

,score_x,gene,product,description,description(VFDB),Cog cat description


### transport

In [10]:
indices = [index for index in selected_features_final.index if 'transport' in selected_features_final.loc[index, 'product'].lower() or 'transport' in selected_features_final.loc[index, 'Subsystem(s)'].lower()]
selected_features_final.loc[indices][cols]

,score_x,gene,product,description,description(VFDB),Cog cat description
4,89.012527,mntA,Manganese-binding lipoprotein MntA,Periplasmic-binding protein,,Inorganic ion transport and metabolism
25,-44.611851,,Maltose transport system permease protein MalF,Maltose transporter membrane protein,,Carbohydrate metabolism and transport
35,-37.292459,ydcV,Inner membrane ABC transporter permease protei...,Putrescine ABC transporter,,Inorganic ion transport and metabolism


### metabolism

In [67]:
cols = ['Allele ID','score_x','gene', 'product', 'description', 'description(VFDB)', 'Cog cat description', 'COG (grouped)', 'Reaction(s)', 'Subsystem(s)', 'Module(s)','gene_name(model)','locus_tag_y']
indices_metabolism = set(selected_features_final.loc[selected_features_final['COG (grouped)'] == 'Metabolism'].index.tolist() + selected_features_final.loc[selected_features_final['Reaction(s)'] != ''].index.tolist())
selected_features_final.loc[indices_metabolism, 'Category (curated)'] = 'Metabolism'
selected_features_final = selected_features_final.fillna('')
print(len(indices_metabolism))
selected_features_final.loc[indices_metabolism ][cols].sort_values(by = 'score_x')


14


,Allele ID,score_x,gene,product,description,description(VFDB),Cog cat description,COG (grouped),Reaction(s),Subsystem(s),Module(s),gene_name(model),locus_tag_y
0,Allele_66792,-129.278267,proC_1,Pyrroline-5-carboxylate reductase,Prephenate dehydrogenase,,Amino acid metabolism and transport,Metabolism,prephenate dehydrogenase AND prephenate dehydr...,Amino Acid Metabolism,"Phenylalanine, Tyrosine and Tryptophan Metabolism",tyrA,USA300HOU_RS06950
6,Allele_58753,-75.129846,murI,Glutamate racemase,Provides the (R)-glutamate required for cell w...,,Cell wall structure and biogenesis,Cellular Processes & Signaling,glutamate racemase,Cell Wall/Membrane/Envelope Metabolism,Peptidoglycan metabolism,murI,USA300HOU_RS05755
8,Allele_79102,-70.867660,trpE,Anthranilate synthase component 1,anthranilate synthase component I,,Amino acid metabolism and transport,Metabolism,anthranilate synthase AND Anthranilate synthas...,Amino Acid Metabolism,"Phenylalanine, Tyrosine and Tryptophan Metabolism",trpE,USA300HOU_RS06960
9,Allele_45236,-70.734897,pyrR,Bifunctional protein PyrR,Bifunctional pyrimidine regulatory protein Pyr...,,Nucleotide metabolism,Metabolism,uracil phosphoribosyltransferase,Nucleotide Metabolism,Pyrimidine metabolism,pyrR,USA300HOU_RS06020
13,Allele_13414,-62.234908,pepA_2,Glutamyl aminopeptidase,aminopeptidase,,Amino acid metabolism and transport,Metabolism,,,,,
17,Allele_77068,-52.100680,murG,UDP-N-acetylglucosamine--N-acetylmuramyl- (pen...,Cell wall formation. Catalyzes the transfer of...,,Cell wall structure and biogenesis,Cellular Processes & Signaling,UDP-N-acetylglucosamine--N-acetylmuramyl-(pent...,Cell Wall/Membrane/Envelope Metabolism,Peptidoglycan metabolism,murG,USA300HOU_RS07245
22,Allele_13547,-46.533798,asnS,Asparagine--tRNA ligase,asparaginyl-tRNA synthetase,,"Translation, including ribosome structure and ...",Information Storage & Processing,Asparaginyl-tRNA synthetase,Nucleotide Metabolism,tRNA Charging,asnS,USA300HOU_RS07440
25,Allele_2405,-44.611851,,Maltose transport system permease protein MalF,Maltose transporter membrane protein,,Carbohydrate metabolism and transport,Metabolism,Maltotriose transport via ABC system AND Malto...,Transport,Alternative carbon metabolism,malF,USA300HOU_RS01100
35,Allele_56669,-37.292459,ydcV,Inner membrane ABC transporter permease protei...,Putrescine ABC transporter,,Inorganic ion transport and metabolism,Metabolism,AND Spermidine transport via ABC system AND P...,Transport,Transport AND Spermidine catabolism,potC,USA300HOU_RS05495
20,Allele_65512,48.686675,arnC,Undecaprenyl-phosphate 4-deoxy-4-formamido-L-a...,Glycosyl transferase,,Cell wall structure and biogenesis,Cellular Processes & Signaling,poly(ribitol phosphate) teichoic acid alpha-O-...,Cell Wall/Membrane/Envelope Metabolism,Wall teichoic acid metabolism,tarS,USA300HOU_RS01340


In [71]:
for rx in model.genes.USA300HOU_RS09140.reactions:
    print(rx, rx.name, rx.notes)

PSERT: 3php_c + glu__L_c --> akg_c + pser__L_c phosphoserine transaminase {'Module': 'Serine, cysteine, glycine and methionine biosynthesis', 'Old_subsystem': 'Glycine and Serine Metabolism'}
NALN6: akg_c + nal2a6o_c --> glu__L_c + n6all26d_c N-acetyl-LL-diaminopimelate aminotransferase {'Module': 'Lysine biosynthesis', 'References': 'http://www.uniprot.org/uniprot/P16524'}


In [13]:
hypothetical_index = {index for index in selected_features_final.index if 'hypothetical protein' in selected_features_final.loc[index, 'product']}
len(hypothetical_index)

11

In [17]:
selected_features_final.shape

(40, 33)

In [14]:
selected_features_final.loc[(selected_features_final['COG (grouped)'] == 'Information Storage & Processing') & (selected_features_final['Category (curated)'] == '')]

,Allele ID,score_x,CD-hit ID,Allele sequence,EC_number,gene,product,VFDB hit,VF_id(VFDB),description(VFDB),...,path,COG cat,description,Cog cat description,COG (grouped),Module(s),Reaction(s),Subsystem(s),locus_tag_y,Category (curated)
10,Allele_21602,69.394567,Cluster 2816,MVSKKKALEMIDVIANMFPDAECELKHDNPFELTIAVLLSAQCTDV...,4.2.99.18,nth_1,Endonuclease III,,,,...,0QI8Q|8.4e-76|254.9,L,endonuclease III,"Replication, recombination and repair",Information Storage & Processing,,,,,
19,Allele_780,-49.759902,Cluster 1922,METIIEEYLRFIQIEKGLSSNTIGAYRRDLKKYQDYMTEHHISHID...,,xerD_1,Tyrosine recombinase XerD,,,,...,0QIWP|3.1e-111|372.4,L,"Site-specific tyrosine recombinase, which acts...","Replication, recombination and repair",Information Storage & Processing,,,,,
29,Allele_32968,-40.983696,Cluster 265,MGKIKELQTSLANKIAAGEVVERPSSVVKELLENAIDAGATEISIE...,,mutL,DNA mismatch repair protein MutL,,,,...,0QHWQ|2.5e-138|463.4,L,This protein is involved in the repair of mism...,"Replication, recombination and repair",Information Storage & Processing,,,,,
32,Allele_3414,39.745381,Cluster 599,MQNFKELGISDNTVQSLESMGFKEPTPIQKDSIPYALQGIDILGQA...,3.6.4.13,cshA,DEAD-box ATP-dependent RNA helicase CshA,,,,...,0QI2F|1.2e-132|443.4,L,ATP-dependent RNA helicase,"Replication, recombination and repair",Information Storage & Processing,,,,,
33,Allele_4709,-38.530957,Cluster 4882,MAVKIRLTRLGSKRNPFYRIVVADARSPRDGRIIEQIGTYNPTSAN...,,rpsP,30S ribosomal protein S16,,,,...,0QTRY|3.2e-25|89.2,J,30s ribosomal protein s16,"Translation, including ribosome structure and ...",Information Storage & Processing,,,,,


In [15]:
selected_features_final.loc[(selected_features_final['COG (grouped)'] == 'Cellular Processes & Signaling') & (selected_features_final['Category (curated)'] == '')]

,Allele ID,score_x,CD-hit ID,Allele sequence,EC_number,gene,product,VFDB hit,VF_id(VFDB),description(VFDB),...,path,COG cat,description,Cog cat description,COG (grouped),Module(s),Reaction(s),Subsystem(s),locus_tag_y,Category (curated)
1,Allele_76691,116.752713,Cluster 181,MDIGKKHVIPKSQYRRKRREFFHNEDREENLNQHQDKQNIDNTTSK...,3.4.21.-,,Serine protease HtrA-like protein,,,,...,0QHTY|6.6e-64|217.0,O,protease,Molecular chaperones and related functions,Cellular Processes & Signaling,,,,,
18,Allele_37402,-49.960479,Cluster 1497,MKIRVVIPCFNEGEVITQTHQQLTEILSQDSSVKGYDYNMLFIDDG...,2.4.-.-,csbB,Putative glycosyltransferase CsbB,,,,...,0QK1D|2.3e-105|352.8,M,Catalyzes the transfer of 4-deoxy-4-formamido-...,Cell wall structure and biogenesis,Cellular Processes & Signaling,,,,,
21,Allele_7081,46.705751,Cluster 69,MVYLKSIDAIGFKSFADQTNVQFDKGVTAIVGPNGSGKSNITDAIK...,,smc_2,Chromosome partition protein Smc,,,,...,0QJVR|1.1e-229|766.5,D,Required for chromosome condensation and parti...,Cell division and chromosome,Cellular Processes & Signaling,,,,,
24,Allele_74190,44.941083,Cluster 177,MAQAKKKSTAKKKTASKKRTNSRKKKNDNPIRYVIAILVVVLMVLG...,,ftsK,DNA translocase FtsK,,,,...,0QHX9|4e-202|674.3,D,Essential cell division protein that coordinat...,Cell division and chromosome,Cellular Processes & Signaling,,,,,
37,Allele_51169,36.198510,Cluster 156,MNNEVPETLKDKRVMSLDMGTVVAGTKYRGEFEERLKKVMEEIQQA...,,clpC,ATP-dependent Clp protease ATP-binding subunit...,,,,...,0QIFG|3.2e-247|823.7,O,Part of a stress-induced multi-chaperone syste...,Molecular chaperones and related functions,Cellular Processes & Signaling,,,,,


In [17]:
selected_features_final.loc[(selected_features_final['COG (grouped)'] == '') & (selected_features_final['Category (curated)'] == '')]

,Allele ID,score_x,CD-hit ID,Allele sequence,EC_number,gene,product,VFDB hit,VF_id(VFDB),description(VFDB),...,path,COG cat,description,Cog cat description,COG (grouped),Module(s),Reaction(s),Subsystem(s),locus_tag_y,Category (curated)
3,Allele_77903,89.774965,Cluster 4123,MSASLYIAIILVIAIIAYMIVQQILNKRAVKELDQNEFHNGIRKAQ...,,yibN,putative protein YibN,,,,...,,,,,,,,,,
5,Allele_72124,-87.133847,Cluster 5066,MAGKIAKYFAMSFAILNVVSEPRVINNCFPIRTTSISLVGLESKSI...,,,hypothetical protein,,,,...,,,,,,,,,,
7,Allele_37502,-71.453375,Cluster 2922,MVDLASSWIVFFLPMINWFIPKNYVKISKEEFDVLNIFKPVRNNAF...,,,hypothetical protein,,,,...,,,,,,,,,,
12,Allele_42990,-62.990927,Cluster 867,MKNKDYTINPGNLKSNSEATSSISSVSYEIENANNHGLKKDKIHDQ...,,,hypothetical protein,,,,...,,,,,,,,,,
14,Allele_42258,-61.729178,Cluster 2135,MPKITKIEVQKKNKERFNLFLDEQFEMGIDIDTLVKFNLKKGQQLE...,,recX,Regulatory protein RecX,,,,...,,,,,,,,,,
16,Allele_86404,-54.860748,Cluster 4383,MTLYEDVKLLLKKNGVEVKSDEEEIFKMEVDGILEDVRDITNNDFM...,,,hypothetical protein,,,,...,,,,,,,,,,
23,Allele_73002,46.478289,Cluster 5086,MGCGPQHREIGFPISTDNASWRGPNTEADEKSAYNNVQVGDGPQQR...,,,hypothetical protein,,,,...,,,,,,,,,,
27,Allele_87042,42.180648,Cluster 326,MAXXXXGNRRLEKPKGIMIRNAQTMSSVESLYNSRKKYKQDVEYPH...,,,Bifunctional autolysin,,,,...,,,,,,,,,,
28,Allele_4769,-41.225590,Cluster 1174,MPTLYELKQSLGMIGQQLKNKNDELSQKATDPNIDMEDIKQLETEK...,,,hypothetical protein,,,,...,,,,,,,,,,
30,Allele_46480,-40.892422,Cluster 3401,MKKTLTLNSLKRNEEFLKFYEKFLFEELDLGEQKKILSLIVLFLNA...,,,hypothetical protein,,,,...,,,,,,,,,,


In [16]:
selected_features_final.loc[set(selected_features_final.index) - indices_metabolism][cols]

,Allele ID,score_x,gene,product,description,description(VFDB),Cog cat description,COG (grouped),Reaction(s),Subsystem(s),Module(s)
1,Allele_76691,116.752713,,Serine protease HtrA-like protein,protease,,Molecular chaperones and related functions,Cellular Processes & Signaling,,,
3,Allele_77903,89.774965,yibN,putative protein YibN,,,,,,,
5,Allele_72124,-87.133847,,hypothetical protein,,,,,,,
7,Allele_37502,-71.453375,,hypothetical protein,,,,,,,
10,Allele_21602,69.394567,nth_1,Endonuclease III,endonuclease III,,"Replication, recombination and repair",Information Storage & Processing,,,
12,Allele_42990,-62.990927,,hypothetical protein,,,,,,,
14,Allele_42258,-61.729178,recX,Regulatory protein RecX,,,,,,,
16,Allele_86404,-54.860748,,hypothetical protein,,,,,,,
18,Allele_37402,-49.960479,csbB,Putative glycosyltransferase CsbB,Catalyzes the transfer of 4-deoxy-4-formamido-...,,Cell wall structure and biogenesis,Cellular Processes & Signaling,,,
19,Allele_780,-49.759902,xerD_1,Tyrosine recombinase XerD,"Site-specific tyrosine recombinase, which acts...",,"Replication, recombination and repair",Information Storage & Processing,,,
